In [39]:
import json
import logging
import os
import shutil
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
import torch
import numpy as np
from torch.utils.data import DataLoader, DistributedSampler
from huggingface_hub import hf_hub_download
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import Dataset

# تنظیم لاگینگ
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# تشخیص دستگاه به صورت خودکار
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# پیام مناسب برای دستگاه انتخاب‌شده
if device.type == "cuda":
    logger.info(f"Using device: {torch.cuda.get_device_name(0)}")
    logger.info(f"Total available GPUs: {torch.cuda.device_count()}")
    torch.cuda.set_device(0)  # تنظیم GPU شماره 0
else:
    logger.info("CUDA device not available, falling back to CPU")

# تعیین تعداد نخ‌ها برای پردازش
torch.set_num_threads(4)

# نمایش دستگاه انتخاب‌شده
logger.info(f"Final device: {device}")


2025-01-16 19:58:18,851 - __main__ - INFO - Using device: Tesla K80
2025-01-16 19:58:18,853 - __main__ - INFO - Total available GPUs: 4
2025-01-16 19:58:18,855 - __main__ - INFO - Final device: cuda


In [40]:
import transformers  # noqa: E402
from transformers import AutoConfig, AutoTokenizer, HfArgumentParser  # noqa: E402

In [41]:
from collections import defaultdict
from copy import deepcopy
from dataclasses import dataclass, field
import importlib
import inspect
import itertools
import logging
import time
from typing import Dict, Tuple, Union, Optional

import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from transformers.optimization import get_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
# import horovod.torch as hvd

# from lm_experiments_tools.utils import rank_0
# import horovod.torch as hvd


logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


In [42]:
import importlib
import os
import platform
import subprocess
import functools


import torch
import transformers

def get_cls_by_name(name: str) -> type:
    """Get class by its name and module path.

    Args:
        name (str): e.g., transfomers:T5ForConditionalGeneration, modeling_t5:my_class

    Returns:
        type: found class for `name`
    """
    module_name, cls_name = name.split(':')
    return getattr(importlib.import_module(module_name), cls_name)


def get_git_hash_commit() -> str:
    return subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()

def collect_run_configuration(args, env_vars=['CUDA_VISIBLE_DEVICES']):
    args_dict = dict(vars(args))
    args_dict['ENV'] = {}
    for env_var in env_vars:
        args_dict['ENV'][env_var] = os.environ.get(env_var, '')
    # args_dict['HVD_INIT'] = hvd.is_initialized()
    # if hvd.is_initialized():
    #     args_dict['HVD_SIZE'] = hvd.size()
    args_dict['MACHINE'] = platform.node()
    args_dict['COMMIT'] = get_git_hash_commit()
    return args_dict


In [43]:
#avoiding from exploding gradient
if args.clip_grad_norm is not None and args.clip_grad_value is not None:
    raise RuntimeError(f'Only one from clip_grad_norm and clip_grad_value should be set, but found '
                        f'clip_grad_norm = {args.clip_grad_norm}, '#usually use this with value of 1 (python script.py --clip_grad_norm 1.0)
                        f'clip_grad_value = {args.clip_grad_value}.')
clip_grad = False
if args.clip_grad_norm or args.clip_grad_value:
    clip_grad = True

In [44]:
#in this setting model tries to minimize loss as default
args.optimize_mode = getattr(args, 'optimize_mode', 'min')
args.optimize_metric = getattr(args, 'optimize_metric', 'loss')
if args.optimize_mode == 'min':
    metric_improved_fn = lambda old_m, new_m: old_m > new_m
else:
    metric_improved_fn = lambda old_m, new_m: old_m < new_m
early_stopping_counter = 0

In [45]:
tb = None #???
if args.model_path is not None:
    tb = SummaryWriter(log_dir=args.model_path)


# move model to gpu
model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [46]:
args.lr_scheduler = None

In [47]:

if args.lr_scheduler: #scheduler name from transformers.optimization: linear, cosine, cosine_with_restarts, 'polynomial, constant, constant_with_warmup (default: None)'
    if args.lr is None:
        raise RuntimeError('Set learning_rate to use learning rate schedulers.')
    if args.num_training_steps is None:
        args.num_training_steps = args.iters
    lr_scheduler = get_scheduler(args.lr_scheduler, optimizer,
                                        args.num_warmup_steps, args.num_training_steps)
else:
    lr_scheduler = None

args.use_lr_drop = getattr(args, 'use_lr_drop', False) #use ReduceLROnPlateau  as scheduler or not(default = false)
if args.use_lr_drop and lr_scheduler is not None: #one scheduler can be used
    raise RuntimeError('lr drop can not be used with other lr schedulers')
if args.use_lr_drop and valid_dataloader is None:
    raise RuntimeError('lr drop is based on validation metrics, but validation set is not set')
if args.use_lr_drop: #to reduce lr if loss doesn't improve
    lr_drop_scheduler = ReduceLROnPlateau(optimizer, mode=args.optimize_mode,
                                                factor=args.lr_drop_factor,
                                                patience=args.lr_drop_patience,
                                                threshold=args.lr_drop_threshold,
                                                threshold_mode=args.lr_drop_threshold_mode,
                                                cooldown=args.lr_drop_cooldown,
                                                min_lr=args.lr_drop_min_lr,
                                                eps=args.lr_drop_eps,
                                                verbose=True)
else:
    lr_drop_scheduler = None

In [48]:
# Apex
if args.fp16:
    try:
        amp = importlib.import_module('apex.amp')
    except ImportError:
        raise ImportError('Install NVIDIA APEX to use fp16 training! Check README.md for instructions.')
    model, optimizer = amp.initialize(model, optimizer,
                                                        enabled=args.fp16, opt_level=args.apex_opt_lvl,
                                                        min_loss_scale=args.min_loss_scale,
                                                        )


In [49]:
model_forward_args = set(inspect.getfullargspec(model.forward).args)#batchsize for all gpu

In [50]:
batch_size = args.batch_size
is_train_mode=True
batch_transform_fn=None
if is_train_mode:
    model.train()
    optimizer.zero_grad()
else:
    model.eval()

if batch_transform_fn:
    batch = batch_transform_fn(batch)
for k in batch:
    if k in model_forward_args:
        batch[k] = batch[k].cuda()

batch_metrics = defaultdict(lambda: 0.0)
batch_metrics_data = defaultdict(lambda: [])

NameError: name 'batch' is not defined

In [51]:
batch_metrics_fn=lambda _, y: {'loss': y['loss']}

In [52]:
def metrics_fn(data):
    # compute metrics based on stored labels, predictions, ...
    metrics = {}
    y, p = None, None

    if args.model_type == 'encoder-decoder' and 'generation_outputs' in data:
        # replace -100 with pad token in labels
        y = data['labels']
        # print('!', data['generation_outputs'].shape)
        p = tokenizer.batch_decode(data['generation_outputs'], skip_special_tokens=True)
        if args.show_valid_examples > 0:
        # if args.show_valid_examples > 0:
            for i in range(min(args.show_valid_examples, len(y))):
                logger.info(f'y: {y[i]}')
                logger.info(f'p: {p[i]}')
                logger.info(f'p ids: {data["generation_outputs"][i]}')
                logger.info('-' * 50)


    if y is not None and p is not None:
        metrics['exact_match'] = accuracy_score(y, p) * 100

    return metrics

In [53]:
def keep_for_metrics_fn(batch, output):
    # select data from batch and model output that would be used to compute metrics
    data = {}
    if 'generation_outputs' in output:
        data['labels'] = batch['target_text']  # برچسب‌های اصلی (متن هدف)
        data['generation_outputs'] = output['generation_outputs']  # متن تولیدشده توسط مدل
    return data

In [208]:
batch_metrics = defaultdict(lambda: 0.0)
batch_metrics_data = defaultdict(lambda: [])
for j in range(0, len(batch['input_ids']), 1):
    subbatch = {k: batch[k][j: j + 1] for k in batch}
    #print(subbatch)
    outputs =model(**{k: subbatch[k] for k in subbatch if k in model_forward_args})
    loss = outputs['loss']
    #print(outputs)
    metrics = batch_metrics_fn(subbatch, outputs)
    #print(metrics)
    loss = loss / 1
    for k in metrics:
        metrics[k] = metrics[k] /1
        batch_metrics[k] += metrics[k].detach().item()
        print(batch_metrics)
    # if keep_for_metrics_fn and metrics_fn:
    #     # for k, v in keep_for_metrics_fn(subbatch, outputs).items():
    #     #     batch_metrics_data[k] += [v.detach().cpu() if isinstance(v, torch.Tensor) else v]
    #     print(batch_metrics_data)

defaultdict(<function <lambda> at 0x7f27d8328e50>, {'loss': 5.775958061218262})


In [54]:
clip_grad = False
# lr_scheduler = "constant_with_warmup"
batch_transform_fn=None
model_forward_args = set(inspect.getfullargspec(model.forward).args)#batchsize for all gpu

batch_metrics_fn=lambda _, y: {'loss': y['loss']}

def step(batch, is_train_mode=True) -> Tuple[Dict[str, float], Dict[str, list]]:
    """Performs one step (forward and optionally backward and optimizer.step()) over data in a batch.

    Batch is splitted on sub-batches of self.args.batch_size size, loss and gradients are accumulated.

    Args:
        batch (dict): dict with inputs, inputs_mask, targets
        is_train_mode (bool, optional): In train mode we compute gradients, do backprop and optimizer.step().
            Defaults to True.

    Returns:
        float: loss on batch
    """
    batch_size = args.batch_size
    if is_train_mode:
        model.train()
        optimizer.zero_grad()
    else:
        model.eval()

    if batch_transform_fn:
        batch = batch_transform_fn(batch)
    for k in batch:
        if k in model_forward_args:
            batch[k] = batch[k].cuda()

    batch_metrics = defaultdict(lambda: 0.0)
    batch_metrics_data = defaultdict(lambda: [])
    with torch.set_grad_enabled(is_train_mode):
        for j in range(0, len(batch['input_ids']), batch_size):
            subbatch = {k: batch[k][j: j + batch_size] for k in batch}
            # filter items from batch that are not used by model forward
            outputs =model(**{k: subbatch[k] for k in subbatch if k in model_forward_args})
            loss = outputs['loss']

            if not is_train_mode and args.use_generate_on_valid:
                generate_kwargs = deepcopy(generate_kwargs)
                if 'max_length' not in generate_kwargs and 'labels' in subbatch:
                    # if max_length is not set and labels are in subbatch, generate to the length of labels+1
                    # +1 as special tokens could be generated by the model
                    generate_kwargs['max_length'] = subbatch['labels'].shape[-1] + 1
                if 'attention_mask' in subbatch:
                    generate_kwargs['attention_mask'] = subbatch['attention_mask']
                if 'global_attention_mask' in subbatch:
                    generate_kwargs['global_attention_mask'] = subbatch['global_attention_mask']
                generation_outputs = model.generate(subbatch['input_ids'], **generate_kwargs)
                outputs['generation_outputs'] = generation_outputs

            metrics = batch_metrics_fn(subbatch, outputs)
            # divide loss on gradient_accumulation_steps to get average loss for sub-batches
            loss = loss / args.gradient_accumulation_steps
            for k in metrics:
                metrics[k] = metrics[k] /args.gradient_accumulation_steps
                batch_metrics[k] += metrics[k].detach().item()

            if keep_for_metrics_fn and metrics_fn:
                for k, v in keep_for_metrics_fn(subbatch, outputs).items():
                    batch_metrics_data[k] += [v.detach().cpu() if isinstance(v, torch.Tensor) else v]

            if is_train_mode:
                if args.fp16:
                    with amp.scale_loss(loss, optimizer) as scaled_loss:
                        scaled_loss.backward()
                        # last sub-batch, call synchronize within amp.scale_loss scope
                        # mb move to just above with optimizer.skip_synchronize()
                        if j == (len(batch['input_ids']) // batch_size - 1) * batch_size:
                            optimizer.synchronize()
                else:
                    loss.backward()

        if is_train_mode:
            if args.fp16:
                if clip_grad:
                    # grads already in sync
                    _clip_gradients()
                # with self.optimizer.skip_synchronize():
                #     self.optimizer.step()
                scaler.step(optimizer)
                scaler.update()
            else:
                if clip_grad:

                    _clip_gradients()
                    optimizer.step()
                else:
                    optimizer.step()

            if lr_scheduler:
                lr_scheduler.step()
    return batch_metrics, batch_metrics_data


In [55]:
batch

NameError: name 'batch' is not defined

In [59]:

step(batch,is_train_mode=True)

NameError: name 'batch' is not defined

In [78]:
train_dataloader = DataLoader(
train_dataset,
batch_size=per_worker_batch_size,
shuffle=True,
collate_fn=collate_fn,
**kwargs
)
train_dataloader

In [122]:
# مقداردهی اولیه متغیرها
n_iter = 0
n_epoch = 0


# تعریف تابع تولید دسته‌ها
def _train_batch_generator():
    global n_iter, n_epoch  # دسترسی به متغیرهای سراسری
    while n_iter < args.iters:
        for batch in train_dataloader:
            if n_iter >= args.iters:
                return
            yield batch
            n_iter += 1  # افزایش شمارنده تکرارها
        n_epoch += 1  # افزایش شمارنده epoch



In [123]:
def _add_metrics_data(metrics_data: Dict[str, torch.Tensor], split: str):
    """Adds metrics data to keep. These data would be used to compute metrics later with get_metrics.

    Args:
        split (str): train / valid
        metrics_data (Dict[str, torch.Tensor]): dict with metrics data, data[name].shape[0] is batch size.
    """
    for k in metrics_data:
        metrics_data[split][k] += metrics_data[k]

In [124]:
args.log_interval =1 

In [125]:
args.log_interval

1

In [126]:
def collect_metrics(split: str) -> dict:
    """
    collects all metrics from batch_metrics and computes metrics available from metrics_data
    once metrics are collected we drop everything that was collected

    Args:
        split (str): data split name train/valid for which metrics should be collected

    Returns:
        dict: dictionary with collected metrics
    """

    # batch-lvl metrics
    metrics = {}
    for k in batch_metrics[split]:
        # جمع‌آوری و محاسبه میانگین متریک‌ها
        metrics[k] = batch_metrics[split][k]
        metrics[k] = np.mean(metrics[k])

    # compute metrics from metrics data
    if keep_for_metrics_fn and metrics_fn:
        metrics_data = {}
        for k in metrics_data[split]:
            metrics_data[k] = list(itertools.chain.from_iterable(metrics_data[split][k]))
            m_shape = getattr(metrics_data[k][0], 'shape', None)
            if m_shape is None:
                metrics_data[k] = list(itertools.chain.from_iterable(metrics_data[k]))
            elif len(m_shape) == 0:
                metrics_data[k] = torch.stack(metrics_data[k])
            elif all(m_shape[1:] == t.shape[1:] for t in metrics_data[k]):
                metrics_data[k] = torch.cat(metrics_data[k])
            else:
                metrics_data[k] = list(itertools.chain.from_iterable([t.tolist() for t in metrics_data[k]]))
        m = metrics_fn(metrics_data)
        if len(metrics.keys() & m.keys()) != 0:
            _log_warning(f'metrics ({m.keys()}) and batch-lvl metrics ({metrics.keys()}) have common names. '
                                f'Batch-lvl metric value would be overwritten.')
        metrics.update(m)
    _reset_batch_metrics(split)
    _reset_metrics_data(split)
    return metrics

In [127]:
step(batch, is_train_mode=True)

(defaultdict(<function __main__.step.<locals>.<lambda>()>,
             {'loss': 6.71657133102417}),
 defaultdict(<function __main__.step.<locals>.<lambda>()>, {}))

In [128]:
def _add_batch_metrics(batch_metrics: Dict[str, Union[float, torch.Tensor]], split: str):
    """Adds metrics values for batch-lvl metrics.

    Args:
        split (str): train / valid
        batch_metrics (Dict[str, Union[float, torch.Tensor]]): batch-lvl metrics values, scalars.
    """
    for k in batch_metrics:
        batch_metrics[split][k] += [batch_metrics[k]]


In [129]:
batch_metrics, batch_metrics_data = step(batch, is_train_mode=True)

In [130]:
batch_metrics

defaultdict(<function __main__.step.<locals>.<lambda>()>,
            {'loss': 6.816182613372803})

In [131]:
batch_metrics["train"] = defaultdict(lambda: [])

batch_metrics["train"]

defaultdict(<function __main__.<lambda>()>, {})

In [132]:
_reset_batch_metrics('train')

In [133]:
collect_metrics(split='train')

KeyError: 'train'

In [119]:
_reset_batch_metrics('train')
_reset_metrics_data('train')
best_valid_metric = np.inf if args.optimize_mode == 'min' else -np.inf
valid_metric = best_valid_metric
valid_loss = np.inf
train_loss = np.inf
early_stopping_counter = 0
for batch in train_batches:
    iteration_start = time.time()
    batch_metrics, batch_metrics_data = step(batch, is_train_mode=True)
    print(batch_metrics)
    iteration_time = time.time() - iteration_start
    _add_batch_metrics(batch_metrics, split='train')
    if keep_for_metrics_fn and metrics_fn:
        _add_metrics_data(batch_metrics_data, split='train')

                # logging
    if args.log_interval and n_iter % args.log_interval == 0:
        # batch-lvl averaged metrics:
        train_metrics = collect_metrics(split='train')
        train_loss = train_metrics['loss']
        print(train_loss)

In [73]:
batch_metrics = {}

def _reset_batch_metrics(split=None):
    global batch_metrics  # استفاده از متغیر سراسری
    if split is None:
        batch_metrics = {}
        batch_metrics['train'] = defaultdict(lambda: [])
        batch_metrics['valid'] = defaultdict(lambda: [])
    else:
        batch_metrics[split] = defaultdict(lambda: [])


In [96]:
metrics_data = {}
def _reset_metrics_data(split=None):
    global metrics_data
    if split is None:
        metrics_data = {}
        metrics_data['train'] = defaultdict(lambda: [])
        metrics_data['valid'] = defaultdict(lambda: [])
    else:
        metrics_data[split] = defaultdict(lambda: [])


In [16]:
class Trainer:
    def __init__(self, args, model, optimizer, train_dataloader, valid_dataloader,
                 batch_transform_fn=None,
                 batch_metrics_fn=lambda _, y: {'loss': y['loss']},
                 keep_for_metrics_fn=None,
                 metrics_fn=None,
                 generate_kwargs={},
                 ) -> None:
        """Implements training loop with horovod multi-gpu, apex fp16 & grad accumulation support.

        Args:
            args: TrainerArgs passed from CLI
            model: torch model to train, model is compatible with HF interfaces
            optimizer: torch optimizer
            train_dataloader (torch.utils.data.DataLoader): train set torch dataloader, distributed-aware.
            valid_dataloader (Optional(torch.utils.data.DataLoader)]): validation set torch dataloader,
                distributed-aware, optional.
            batch_transform_fn (Optional): function to be applied to the output from DataLoader, should be used to
                create inputs compatible (if not already) with HF model, e.g.:
                    {'input_ids': ..., 'attention_mask': ..., 'labels': ..., ...}.
            batch_metrics_fn (Optional): function to be applied to model outputs to compute batch-lvl metrics, metrics
                are averaged across batches: avg_i(metric(batch_i, labels_i)),
                not metric([batch_1; batch_2; ...], labels). Could be used for computing loss, metrics on large
                datasets, pre-training, where exact metrics values are not so important or computing exact metrics
                is resource-exhaustive.
            keep_for_metrics_fn (Optional): f(batch, outputs) to keep predictions, labels or other data that would be
                used to compute metrics on full validation set and every log_interval on train set
            metrics_fn (Optional): f(metrics_data) to compute metrics based on values stored by keep_for_metrics_fn
        """
        # we assume that train/valid dataloader are already multi-gpu aware
        self.model = model
        self.optimizer = optimizer
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.batch_transform_fn = batch_transform_fn
        self.batch_metrics_fn = batch_metrics_fn
        self.keep_for_metrics_fn = keep_for_metrics_fn
        self.metrics_fn = metrics_fn
        self.generate_kwargs = generate_kwargs
        self.args = args
        self.per_worker_batch_size = self.args.batch_size * self.args.gradient_accumulation_steps #batchsize for 1 gpu
        self.model_forward_args = set(inspect.getfullargspec(self.model.forward).args)#batchsize for all gpu

        #avoiding from exploding gradient
        if self.args.clip_grad_norm is not None and self.args.clip_grad_value is not None:
            raise RuntimeError(f'Only one from clip_grad_norm and clip_grad_value should be set, but found '
                               f'clip_grad_norm = {self.args.clip_grad_norm}, '#usually use this with value of 1 (python script.py --clip_grad_norm 1.0)
                               f'clip_grad_value = {self.args.clip_grad_value}.')
        self.clip_grad = False
        if self.args.clip_grad_norm or self.args.clip_grad_value:
            self.clip_grad = True

        #in this setting model tries to minimize loss as default
        self.args.optimize_mode = getattr(self.args, 'optimize_mode', 'min')
        self.args.optimize_metric = getattr(self.args, 'optimize_metric', 'loss')
        if self.args.optimize_mode == 'min':
            self.metric_improved_fn = lambda old_m, new_m: old_m > new_m
        else:
            self.metric_improved_fn = lambda old_m, new_m: old_m < new_m
        self.early_stopping_counter = 0


        self.tb = None #???
        if self.args.model_path is not None:
            self.tb = SummaryWriter(log_dir=self.args.model_path)


        # move model to gpu
        self.model.cuda()



        if args.lr_scheduler: #scheduler name from transformers.optimization: linear, cosine, cosine_with_restarts, 'polynomial, constant, constant_with_warmup (default: None)'
            if args.lr is None:
                raise RuntimeError('Set learning_rate to use learning rate schedulers.')
            if args.num_training_steps is None:
                args.num_training_steps = args.iters
            self.lr_scheduler = get_scheduler(args.lr_scheduler, self.optimizer,
                                              args.num_warmup_steps, args.num_training_steps)
        else:
            self.lr_scheduler = None

        self.args.use_lr_drop = getattr(self.args, 'use_lr_drop', False) #use ReduceLROnPlateau  as scheduler or not(default = false)
        if self.args.use_lr_drop and self.lr_scheduler is not None: #one scheduler can be used
            raise RuntimeError('lr drop can not be used with other lr schedulers')
        if self.args.use_lr_drop and self.valid_dataloader is None:
            raise RuntimeError('lr drop is based on validation metrics, but validation set is not set')
        if self.args.use_lr_drop: #to reduce lr if loss doesn't improve
            self.lr_drop_scheduler = ReduceLROnPlateau(self.optimizer, mode=self.args.optimize_mode,
                                                       factor=self.args.lr_drop_factor,
                                                       patience=self.args.lr_drop_patience,
                                                       threshold=self.args.lr_drop_threshold,
                                                       threshold_mode=self.args.lr_drop_threshold_mode,
                                                       cooldown=self.args.lr_drop_cooldown,
                                                       min_lr=self.args.lr_drop_min_lr,
                                                       eps=self.args.lr_drop_eps,
                                                       verbose=True)
        else:
            self.lr_drop_scheduler = None

        # Apex
        if args.fp16:
            try:
                self.amp = importlib.import_module('apex.amp')
            except ImportError:
                raise ImportError('Install NVIDIA APEX to use fp16 training! Check README.md for instructions.')
            self.model, self.optimizer = self.amp.initialize(self.model, self.optimizer,
                                                             enabled=self.args.fp16, opt_level=self.args.apex_opt_lvl,
                                                             min_loss_scale=self.args.min_loss_scale,
                                                             )

        #without chekpoint
        self.n_iter = 0
        self.n_epoch = 0
        self._reset_batch_metrics()
        self._reset_metrics_data()
        #with checkpoint
        if self.args.init_checkpoint:
            self.load(
                args.init_checkpoint,
                self.args.reset_optimizer,
                self.args.reset_lr,
                self.args.reset_iteration
            )

    def step(self, batch, is_train_mode=True) -> Tuple[Dict[str, float], Dict[str, list]]:
        """Performs one step (forward and optionally backward and optimizer.step()) over data in a batch.

        Batch is splitted on sub-batches of self.args.batch_size size, loss and gradients are accumulated.

        Args:
            batch (dict): dict with inputs, inputs_mask, targets
            is_train_mode (bool, optional): In train mode we compute gradients, do backprop and optimizer.step().
                Defaults to True.

        Returns:
            float: loss on batch
        """
        batch_size = self.args.batch_size
        if is_train_mode:
            self.model.train()
            self.optimizer.zero_grad()
        else:
            self.model.eval()

        if self.batch_transform_fn:
            batch = self.batch_transform_fn(batch)
        for k in batch:
            if k in self.model_forward_args:
                batch[k] = batch[k].cuda()

        batch_metrics = defaultdict(lambda: 0.0)
        batch_metrics_data = defaultdict(lambda: [])
        with torch.set_grad_enabled(is_train_mode):
            for j in range(0, len(batch['input_ids']), batch_size):
                subbatch = {k: batch[k][j: j + batch_size] for k in batch}
                # filter items from batch that are not used by model forward
                outputs = self.model(**{k: subbatch[k] for k in subbatch if k in self.model_forward_args})
                loss = outputs['loss']

                if not is_train_mode and self.args.use_generate_on_valid:
                    generate_kwargs = deepcopy(self.generate_kwargs)
                    if 'max_length' not in generate_kwargs and 'labels' in subbatch:
                        # if max_length is not set and labels are in subbatch, generate to the length of labels+1
                        # +1 as special tokens could be generated by the model
                        generate_kwargs['max_length'] = subbatch['labels'].shape[-1] + 1
                    if 'attention_mask' in subbatch:
                        generate_kwargs['attention_mask'] = subbatch['attention_mask']
                    if 'global_attention_mask' in subbatch:
                        generate_kwargs['global_attention_mask'] = subbatch['global_attention_mask']
                    generation_outputs = self.model.generate(subbatch['input_ids'], **generate_kwargs)
                    outputs['generation_outputs'] = generation_outputs

                metrics = self.batch_metrics_fn(subbatch, outputs)
                # divide loss on gradient_accumulation_steps to get average loss for sub-batches
                loss = loss / self.args.gradient_accumulation_steps
                for k in metrics:
                    metrics[k] = metrics[k] / self.args.gradient_accumulation_steps
                    batch_metrics[k] += metrics[k].detach().item()

                if self.keep_for_metrics_fn and self.metrics_fn:
                    for k, v in self.keep_for_metrics_fn(subbatch, outputs).items():
                        batch_metrics_data[k] += [v.detach().cpu() if isinstance(v, torch.Tensor) else v]

                if is_train_mode:
                    if self.args.fp16:
                        with self.amp.scale_loss(loss, self.optimizer) as scaled_loss:
                            scaled_loss.backward()
                            # last sub-batch, call synchronize within amp.scale_loss scope
                            # mb move to just above with optimizer.skip_synchronize()
                            if j == (len(batch['input_ids']) // batch_size - 1) * batch_size:
                                self.optimizer.synchronize()
                    else:
                        loss.backward()

            if is_train_mode:
                if self.args.fp16:
                    if self.clip_grad:
                        # grads already in sync
                        self._clip_gradients()
                    # with self.optimizer.skip_synchronize():
                    #     self.optimizer.step()
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                else:
                    if self.clip_grad:

                        self._clip_gradients()
                        self.optimizer.step()
                    else:
                        self.optimizer.step()

                if self.lr_scheduler:
                    self.lr_scheduler.step()
        return batch_metrics, batch_metrics_data

    
    def _train_batch_generator(self):
        while self.n_iter <= self.args.iters:
            # self.train_dataloader
            for batch in self.train_dataloader:
                if self.n_iter > self.args.iters:
                    return
                yield batch
                self.n_iter += 1
            self.n_epoch += 1


    def _skip_n_train_batches(self, train_batches, n):
    # لاگ کردن تعداد Batch‌هایی که قرار است رد شوند
        self._log_info(f'Skipping {n} batches from the dataset from epoch {self.n_epoch}...')
        # Skip کردن Batch‌ها
        for _ in tqdm(itertools.islice(train_batches, n), desc='Skipping...', total=n):
            pass


    def _add_batch_metrics(self, batch_metrics: Dict[str, Union[float, torch.Tensor]], split: str):
        """Adds metrics values for batch-lvl metrics.

        Args:
            split (str): train / valid
            batch_metrics (Dict[str, Union[float, torch.Tensor]]): batch-lvl metrics values, scalars.
        """
        for k in batch_metrics:
            self.batch_metrics[split][k] += [batch_metrics[k]]


    def _add_metrics_data(self, metrics_data: Dict[str, torch.Tensor], split: str):
        """Adds metrics data to keep. These data would be used to compute metrics later with get_metrics.

        Args:
            split (str): train / valid
            metrics_data (Dict[str, torch.Tensor]): dict with metrics data, data[name].shape[0] is batch size.
        """
        for k in metrics_data:
            self.metrics_data[split][k] += metrics_data[k]

    def _reset_batch_metrics(self, split=None):
        if split is None:
            self.batch_metrics = {}
            self.batch_metrics['train'] = defaultdict(lambda: [])
            self.batch_metrics['valid'] = defaultdict(lambda: [])
        else:
            self.batch_metrics[split] = defaultdict(lambda: [])

    def _reset_metrics_data(self, split=None):
        if split is None:
            self.metrics_data = {}
            self.metrics_data['train'] = defaultdict(lambda: [])
            self.metrics_data['valid'] = defaultdict(lambda: [])
        else:
            self.metrics_data[split] = defaultdict(lambda: [])

    @staticmethod
    def _log_info(msg, *args, **kwargs):
        logger.info(msg, *args, **kwargs)

    @staticmethod
    def _log_warning(msg, *args, **kwargs):
        logger.warning(msg, *args, **kwargs)


    def collect_metrics(self, split: str) -> dict:
        """
        collects all metrics from batch_metrics and computes metrics available from metrics_data
        once metrics are collected we drop everything that was collected

        Args:
            split (str): data split name train/valid for which metrics should be collected

        Returns:
            dict: dictionary with collected metrics
        """

        # batch-lvl metrics
        metrics = {}
        for k in self.batch_metrics[split]:
            # جمع‌آوری و محاسبه میانگین متریک‌ها
            #metrics[k] = list(itertools.chain.from_iterable(self.batch_metrics[split][k]))
            metrics[k] = self.batch_metrics[split][k]
            metrics[k] = np.mean(metrics[k])

        # compute metrics from metrics data
        if self.keep_for_metrics_fn and self.metrics_fn:
            metrics_data = {}
            for k in self.metrics_data[split]:
                metrics_data[k] = list(itertools.chain.from_iterable(self.metrics_data[split][k]))
                m_shape = getattr(metrics_data[k][0], 'shape', None)
                if m_shape is None:
                    metrics_data[k] = list(itertools.chain.from_iterable(metrics_data[k]))
                elif len(m_shape) == 0:
                    metrics_data[k] = torch.stack(metrics_data[k])
                elif all(m_shape[1:] == t.shape[1:] for t in metrics_data[k]):
                    metrics_data[k] = torch.cat(metrics_data[k])
                else:
                    metrics_data[k] = list(itertools.chain.from_iterable([t.tolist() for t in metrics_data[k]]))
            m = self.metrics_fn(metrics_data)
            if len(metrics.keys() & m.keys()) != 0:
                self._log_warning(f'metrics ({m.keys()}) and batch-lvl metrics ({metrics.keys()}) have common names. '
                                  f'Batch-lvl metric value would be overwritten.')
            metrics.update(m)
        self._reset_batch_metrics(split)
        self._reset_metrics_data(split)
        return metrics

    def train(self) -> None:
        pbar = None
        
        pbar = tqdm(total=self.args.iters, desc='Train')
        pbar.update(self.n_iter)

        train_batches = self._train_batch_generator()

        # skip used data if needed
        if self.args.skip_used_data and self.n_iter > 0:
            train_size = None
            try:
                train_size = len(self.train_dataloader)
            except TypeError as e:
                self._log_info(f"Can't get train_dataloader length:\n{e}")
            # if we know train_size and number of epochs passed -> jump to this epoch and re-iterate over remainders
            skip_iter = self.n_iter % train_size if train_size else self.n_iter
            self.n_iter = (self.n_iter // train_size) * train_size if train_size else 0
            self._skip_n_train_batches(train_batches, skip_iter)

        self._reset_batch_metrics('train')
        self._reset_metrics_data('train')
        best_valid_metric = np.inf if self.args.optimize_mode == 'min' else -np.inf
        valid_metric = best_valid_metric
        valid_loss = np.inf
        train_loss = np.inf
        self.early_stopping_counter = 0
        for batch in train_batches:
            iteration_start = time.time()
            batch_metrics, batch_metrics_data = self.step(batch, is_train_mode=True)
            iteration_time = time.time() - iteration_start
            self._add_batch_metrics(batch_metrics, split='train')
            if self.keep_for_metrics_fn and self.metrics_fn:
                self._add_metrics_data(batch_metrics_data, split='train')

            # logging
            if self.args.log_interval and self.n_iter % self.args.log_interval == 0:
                # batch-lvl averaged metrics:
                train_metrics = self.collect_metrics(split='train')
                train_loss = train_metrics['loss']

   
                # todo: move logging, move to self.log()
                for k in train_metrics:
                    self._log_info(f'step: {self.n_iter}/{self.args.iters} {k}: {train_metrics[k]:.4f}')
                    if self.tb:
                        self.tb.add_scalar(f'{k}/iterations/train', train_metrics[k], self.n_iter)
                        self.tb.add_scalar(f'{k}/samples/train', train_metrics[k],
                                            self.n_iter )
                # log iteration time
                if self.tb:
                    self.tb.add_scalar('time/iterations/per_iter', iteration_time, self.n_iter)
                    self.tb.add_scalar('time/samples/per_iter', iteration_time,
                                        self.n_iter )
                # log learning rate
                for j, param_group in enumerate(self.optimizer.param_groups):
                    # adafactor uses external lr to compute its own lr if scale_parameter is true
                    # adafactor might not have external lr in case if relative_step is used
                    for p in ['lr', 'scaled_lr']:
                        if p in param_group and param_group[p] is not None and self.tb:
                            self.tb.add_scalar(f'{p}/iterations/param_group_{j}', param_group[p], self.n_iter)
                            self.tb.add_scalar(f'{p}/samples/param_group_{j}', param_group[p],
                                                self.n_iter)
                            
            # validation
            if self.valid_dataloader is not None and self.n_iter % self.args.valid_interval == 0:
                # todo: we can use other metrics than loss here
                valid_metrics = self.validate(self.valid_dataloader)
                valid_loss = valid_metrics['loss']
                valid_metric = valid_metrics[self.args.optimize_metric]
                if self.metric_improved_fn(best_valid_metric, valid_metric):
                    best_valid_metric = valid_metric
                    self.early_stopping_counter = 0
                    self._log_info(f'The best {self.args.optimize_metric} metric was improved to: {best_valid_metric}')
                    if self.args.save_best:
                        self.save(self.args.model_path, suffix='best', metrics=valid_metrics)
                else:
                    self.early_stopping_counter += 1
                    self._log_info(f'Metric was not improved for the last #{self.early_stopping_counter} evaluations')
                if self.lr_drop_scheduler:
                    self.lr_drop_scheduler.step(valid_metric)

            # saving model
            if self.args.save_interval and self.n_iter % self.args.save_interval == 0:
                self.save(self.args.model_path)


            pbar.update(1)
            pbar.set_postfix({'train_loss': f'{train_loss:.3f}',
                                'valid_loss': f'{valid_loss:.3f}',
                                f'best_valid_{self.args.optimize_metric}': f'{best_valid_metric:.3f}'
                                })

            if self.args.early_stopping_patience is not None and \
                    self.early_stopping_counter > self.args.early_stopping_patience:
                self._log_info('Early stopping triggered: stopping training...')
                break


        # todo: run validation, call save model?
        pbar.close()
        self._log_info('Done!')

    def validate(self, dataloader, split='valid', write_tb=True) -> Dict[str, float]:
        self._log_info(f'start validation at step {self.n_iter}')

        self._reset_batch_metrics('valid')
        self._reset_metrics_data('valid')
        for batch in tqdm(dataloader, desc='Validation'):
            batch_metrics, batch_metrics_data = self.step(batch, is_train_mode=False)
            self._add_batch_metrics(batch_metrics, split='valid')
            if self.keep_for_metrics_fn and self.metrics_fn:
                self._add_metrics_data(batch_metrics_data, split='valid')

        metrics = self.collect_metrics(split='valid')
        
            # todo: separate logging from validation/training
        for k in metrics:
            self._log_info(f'Validation on {split} {k}: {metrics[k]:.4f}')
            if self.tb and write_tb:
                self.tb.add_scalar(f'{k}/iterations/{split}', metrics[k], self.n_iter)
                self.tb.add_scalar(f'{k}/samples/{split}', metrics[k], self.n_iter )
        return metrics
    
    def load(self, load_path, reset_optimizer=False, reset_lr=False, reset_iteration=False) -> None:
        # todo: if there is checkpoint in model_path load model from the latest checkpoint (init_checkpoint is None)
        checkpoint = torch.load(load_path, map_location='cpu')
        missing_k, unexpected_k = self.model.load_state_dict(checkpoint["model_state_dict"], strict=False)
        if len(missing_k) != 0:
            self._log_info(f'{missing_k} were not loaded from checkpoint! These parameters were randomly initialized.')
        if len(unexpected_k) != 0:
            self._log_info(f'{unexpected_k} were found in checkpoint, but model is not expecting them!')

        if 'optimizer_state_dict' in checkpoint and not reset_optimizer:
            self._log_info('Loading optimizer state_dict from the checkpoint.')
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        if 'lr_scheduler_state_dict' in checkpoint and self.lr_scheduler and not reset_lr:
            # if set reset_lr we do not load lr_scheduler and keep only the new one from __init__
            self._log_info('Loading lr_scheduler state_dict from the checkpoint.')
            self.lr_scheduler.load_state_dict(checkpoint['lr_scheduler_state_dict'])
        if 'amp' in checkpoint and self.args.fp16:
            self.amp.load_state_dict(checkpoint['amp'])
        if not reset_iteration:
            self.n_iter = checkpoint.get('iteration', 0) + 1  # as saved iteration is already performed
            self.n_epoch = checkpoint.get('epoch', 0)

        self._log_info(f'Model was loaded from: {load_path}')
        self._log_info(f'Start iteration = {self.n_iter}')
        if self.lr_scheduler and reset_lr:
            self._log_warning('lr_scheduler is not loaded from the checkpoint. New lr_scheduler is used with starting'
                              ' step (torch.optim.LRScheduler.__init__ last_epoch parameter) = -1.'
                              ' Current iteration number is ignored.')
        if reset_optimizer:
            self._log_info('Optimizer is not loaded from the checkpoint. New optimizer is created.')

    
    def save(self, save_path, suffix='', metrics=None) -> None:
        if save_path is not None:
            if suffix == '':
                save_path = f'{self.args.model_path}/model_{self.n_iter}.pth'
            else:
                save_path = f'{self.args.model_path}/model_{suffix}.pth'
            to_save = {
                       "model_state_dict": self.model.state_dict(),
                       "optimizer_state_dict": self.optimizer.state_dict(),
                       "iteration": self.n_iter,
                       "epoch": self.n_epoch,
                       }
            if metrics:
                to_save['metrics'] = metrics
            if self.args.fp16:
                to_save['amp'] = self.amp.state_dict()
            if self.lr_scheduler:
                to_save['lr_scheduler_state_dict'] = self.lr_scheduler.state_dict()
            torch.save(to_save, save_path)
            self._log_info(f'Model was saved to {save_path}')

In [147]:
args.valid_interval=1

In [6]:
@dataclass
class TrainerArgs:
    model_path: Optional[str] = field(
        default="./runs/$MODEL_NAME/$TASK_NAME/run_1",
        metadata={'help': 'path where to save model (default: None)'})
    log_interval: Optional[int] = field(
        default=None,
        metadata={'help': 'log to report loss, metrics on training data every N batches (default: None)'})
    valid_interval: Optional[int] = field(
        default=1,#None
        metadata={'help': 'log on validation data every N batches (default: None)'})
    save_interval: Optional[int] = field(
        default=None,
        metadata={'help': 'save model every N steps (default: None)'})
    save_best: bool = field(
        default=False,
        metadata={'help': 'Save best checkpoint if validation set is provided (default: False)'})
    use_generate_on_valid: bool = field(
        default=False,
        metadata={'help': 'Use model.generate method when running validation step (default: False)'})
    # load model args
    init_checkpoint: Optional[str] = field(
        default=None,
        metadata={'help': 'path to init checkpoint to load a model from (default: None).'})
    skip_used_data: bool = field(
        default=False,
        metadata={'help': 'skip batches that were already seen by init_checkpoint (default: False)'})
    reset_lr: bool = field(
        default=False,
        metadata={'help': 'Do not load lr_scheduler from checkpoint and setup new lr (default: False)'})
    reset_iteration: bool = field(
        default=False,
        metadata={'help': 'Do not load iteration number from checkpoint and set it to 0 (default: False)'})
    reset_optimizer: bool = field(
        default=False,
        metadata={'help': 'Do not load optimizer from checkpoint and setup a new one. It might help for continuing '
                          'training from ckpt saved from fp16 O2. Otherwise loss spikes might happen (default: False)'})
    # training args
    lr: Optional[float] = field(
        default=1e-4 ,#None
        metadata={'help': 'learning rate (default: None)'})
    batch_size: int = field(
        default=1,
        metadata={'help': 'input batch size for training (default: 1)'})
    iters: int = field(
        default=10,#1
        metadata={'help': 'number of training steps (i.e., gradient updates) (default: 100)'})
    gradient_accumulation_steps: int = field(
        default=1,
        metadata={'help': 'number of batches to accumulate gradients for each worker, it multiplies total batch size.'})
    fp16: bool = field(
        default=False,
        metadata={'help': 'use apex.amp for fp16 training (default: False)'})
    fp16_allreduce: bool = field(
        default=False, metadata={'help': 'use hvd fp16 compression during allreduce (default: False)'})
    apex_opt_lvl: str = field(
        default='O1',
        metadata={'help': 'apex opt level, O1, O2. (default: O1)'})
    min_loss_scale: Optional[float] = field(
        default=None,
        metadata={'help': 'apex min_loss_scale. (default: None)'})
    clip_grad_norm: Optional[float] = field(
        default=None,
        metadata={'help': 'torch.nn.utils.clip_grad_norm_ max_norm parameter. (default: None)'})
    clip_grad_value: Optional[float] = field(
        default=None,
        metadata={'help': 'torch.nn.utils.clip_grad_value_ clip_value parameter. (default: None)'})
    early_stopping_patience: Optional[int] = field(
        default=None,
        metadata={'help': 'stop training if `early_stopping_patience` subsequent evalutations did not improve value of '
                          '`optimize_metric` on validation set (default: None)'})
    # scheduler args
    lr_scheduler: Optional[str] = field(
        default=None,
        metadata={'help': 'scheduler name from transformers.optimization: linear, cosine, cosine_with_restarts, '
                          'polynomial, constant, constant_with_warmup (default: None)'})
    num_warmup_steps: Optional[int] = field(
        default=None,
        metadata={'help': 'number of warming steps to get to lr (default: None)'})
    num_training_steps: Optional[int] = field(
        default=None,
        metadata={'help': 'number of training steps for scheduler, if not set iters will be used (default: None)'})
    # LRReduceOnPlateau args
    use_lr_drop: bool = field(
        default=False,
        metadata={'help': 'Enable ReduceLROnPlateau scheduler in addition to --lr_scheduler (default: False)'})
    lr_drop_factor: float = field(
        default=0.1,
        metadata={'help': 'torch.optim.lr_scheduler.ReduceLROnPlateau drop parameter. (default: 0.1)'})
    lr_drop_patience: int = field(
        default=10,
        metadata={'help': 'torch.optim.lr_scheduler.ReduceLROnPlateau patience parameter. (default: 10)'})
    lr_drop_threshold: float = field(
        default=1e-04,
        metadata={'help': 'torch.optim.lr_scheduler.ReduceLROnPlateau threshold parameter. (default: 1e-04)'})
    lr_drop_threshold_mode: str = field(
        default='rel',
        metadata={'help': 'torch.optim.lr_scheduler.ReduceLROnPlateau threshold_mode parameter. (default: rel)'})
    lr_drop_cooldown: int = field(
        default=0,
        metadata={'help': 'torch.optim.lr_scheduler.ReduceLROnPlateau cooldown parameter. (default: 0)'})
    lr_drop_min_lr: float = field(
        default=0.0,
        metadata={'help': 'torch.optim.lr_scheduler.ReduceLROnPlateau min_lr parameter. (default: 0.0)'})
    lr_drop_eps: float = field(
        default=1e-08,
        metadata={'help': 'torch.optim.lr_scheduler.ReduceLROnPlateau threshold_mode parameter. (default: 1e-08)'})
    # metrics args
    optimize_metric: str = field(
        default='loss',
        metadata={'help': 'metric name to optimize on validation set, save the best model, drop lr (default: loss)'})
    optimize_mode: str = field(
        default='min',
        metadata={'help': 'metric should be minimized (min) or maximized (max) (default: min)'})



In [7]:
trainer = Trainer(args, model, optimizer, train_dataloader, valid_dataloader, 
                keep_for_metrics_fn=keep_for_metrics_fn, metrics_fn=metrics_fn,
                generate_kwargs=generate_kwargs if args.use_generate_on_valid else {})
trainer

NameError: name 'args' is not defined

In [7]:
valid_df = pd.read_csv("/home/ahmadi/sadaf/GraphNeighborLM/Better-together/data-preparation/datasets/wikidata5m/verbalized_valid.csv")
valid_df.head()

,id,verbalization,head,tail,relation,verbalized_tail
0,0,predict [SEP] Fritz Fullriede military rank [SEP],Q5504910,Q157148,P410,major general
1,1,predict [SEP] major general inverse of militar...,Q157148,Q5504910,inverse of P410,Fritz Fullriede
2,2,predict [SEP] Ruszajny located in the administ...,Q7382697,Q554377,P131,Gmina Barczewo
3,3,predict [SEP] Gmina Barczewo inverse of locate...,Q554377,Q7382697,inverse of P131,Ruszajny
4,4,predict [SEP] Brothers in Arms DS instance of ...,Q2734941,Q7889,P31,video game


In [8]:
train_df = pd.read_csv("/home/ahmadi/sadaf/GraphNeighborLM/Better-together/data-preparation/datasets/wikidata5m/verbalized_train.csv")
train_df.head()

,id,verbalization,head,tail,relation,verbalized_tail
0,0,predict [SEP] Kukavičko Lake instance of [SEP],Q6442440,Q23397,P31,lake
1,1,predict [SEP] lake inverse of instance of [SEP],Q23397,Q6442440,inverse of P31,Kukavičko Lake
2,2,"predict [SEP] Pattersonville, Ohio country [SEP]",Q7148490,Q30,P17,United States of America
3,3,predict [SEP] United States of America inverse...,Q30,Q7148490,inverse of P17,"Pattersonville, Ohio"
4,4,predict [SEP] Johnny Hughes place of birth [SEP],Q16145304,Q3752988,P19,Mountbellew


In [9]:
item = {}
row = train_df.iloc[0]
item["input"] = row['verbalization']
item["outputs"] = row['verbalized_tail']
item

{'input': 'predict [SEP] Kukavičko Lake instance of [SEP]', 'outputs': 'lake'}

In [11]:

row = train_df.iloc[1]
item["input"] = row['verbalization']
item["outputs"] = row['verbalized_tail']
item

{'input': 'predict [SEP] lake inverse of instance of [SEP]',
 'outputs': 'Kukavičko Lake'}

In [10]:
parser = HfArgumentParser(TrainerArgs)
parser.add_argument('--task_name', type=str, help='Scrolls task name: "gov_report", "summ_screen_fd", "qmsum", '
                                                  '"narrative_qa", "qasper", "quality", "contract_nli"')
parser.add_argument('--validate_only', action='store_true', default=False,
                    help='Skip training and run only validation. (default: False)')
parser.add_argument('--working_dir', type=str, default='.',
                    help='working dir, should be a dir with t5-experiments repo (default: .)')
parser.add_argument('--seed', type=int, default=42, help='random seed')
parser.add_argument('--show_valid_examples', type=int, default=2,
                    help='how many valid examples to show during training (default: 0)')

parser.add_argument('--input_seq_len', type=int, default=128, help='input sequnce length (default: 128).')
parser.add_argument('--target_seq_len', type=int, default=16, help='target sequnce length, should be set to '
                                                                   'max(len(target))+1 for EOS (default: 16).')
parser.add_argument('--data_n_workers', type=int, default=2, help='number of dataloader workers (default: 2)')

parser.add_argument('--input_prefix', type=str, default='', help='add task prefix to an input string (default: "")')
parser.add_argument('--drop_neighborhood', action='store_true', default=False, 
                    help='not to include neighborhood in model input')
parser.add_argument('--index_path', default=None, type=str, 
                    help='path to index for hits metric')

parser.add_argument('--inference_entities_path', default=None, type=str, 
                    help='path to names of verbalized entities from inference graph')
# model args
parser.add_argument('--from_pretrained', type=str, default="t5-small",help='model name in HF Model Hub (default: "")')
## 
parser.add_argument('--cpt_path', type=str, help='path of checkpoint folder')

parser.add_argument('--model_cfg', type=str, help='path to model configuration file (default: "")')
parser.add_argument('--model_cls', type=str, default='transformers:T5ForConditionalGeneration',
                    help='model class name to use (default:transformers:T5ForConditionalGeneration)')
parser.add_argument('--model_type', type=str, default='encoder-decoder',
                    help='model type, encoder, encoder-decoder, decoder, affects preprocessing '
                         '(default: encoder-decoder)')

# tokenizer
# todo: add wordpiece tokenizers support?
parser.add_argument('--tokenizer', type=str, default=None, help='path or name of pre-trained HF Tokenizer')

# optimizer args
parser.add_argument('--optimizer', type=str, default='AdamW', help='optimizer name: AdamW, Adafactor. (default: AdamW)')
parser.add_argument('--weight_decay', type=float, default=0.0, help='optimizer weight decay (default: 0.0)')
parser.add_argument('--scale_parameter', action='store_true', default=False,
                    help='Adafactor scale_parameter (default: False)')
parser.add_argument('--relative_step', action='store_true', default=False,
                    help='Adafactor relative_step (default: False)')
parser.add_argument('--warmup_init', action='store_true', default=False,
                    help='Adafactor warmup_init (default: False)')



_StoreTrueAction(option_strings=['--warmup_init'], dest='warmup_init', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Adafactor warmup_init (default: False)', metavar=None)

In [11]:
class KGLMDataset(Dataset):
    def __init__(self, df, neighborhood=True):
        self.df = df
        self.neighborhood = neighborhood
        self.length = len(df)

    def __getitem__(self, idx):

        item = {}
        row = self.df.iloc[idx]

        if self.neighborhood:
            item["input"] = row['verbalization']
        else:
            verbalization = row['verbalization']
            inp = '[SEP]'.join(verbalization.split('[SEP]')[:2])
            item["input"] = inp

        item["outputs"] = row['verbalized_tail']
        return item

    def __len__(self):

        return self.length


In [12]:
train_dataset = KGLMDataset(train_df, neighborhood=True)
train_dataset

In [144]:
if __name__ == '__main__':
    import sys
    sys.argv = ['']  # حذف آرگومان‌های اضافی Jupyter
    args = parser.parse_args()
    logger.info('Running in environment.')
    logger.info('Using a single GPU setup.')
    logger.info(f'FP16 training is set to: {args.fp16}') #default = false
    if args.model_path is None: # path where to save model, default: None
      logger.warning('model_path is not set: config, logs and checkpoints will not be saved.')
    
    if args.model_path is not None:
        model_path = Path(args.model_path)
        if not model_path.exists():
                Path(model_path).mkdir(parents=True)

        args_dict = collect_run_configuration(args)
        # todo: if model path exists and there is config file, write new config file aside
        json.dump(args_dict, open(model_path/'config.json', 'w'), indent=4)

2025-01-16 10:05:53,220 - __main__ - INFO - Running in environment.
2025-01-16 10:05:53,222 - __main__ - INFO - Using a single GPU setup.
2025-01-16 10:05:53,223 - __main__ - INFO - FP16 training is set to: False


In [16]:
args_dict = collect_run_configuration(args)
# todo: if model path exists and there is config file, write new config file aside
json.dump(args_dict, open(model_path/'config.json', 'w'), indent=4)


In [17]:
if args.tokenizer:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer) #sepcialized tokenizer
else:
    tokenizer = AutoTokenizer.from_pretrained(args.from_pretrained) #general tokenizer-->t5-small

/home/ahmadi/sadaf/GraphNeighborLM/sadafenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
tokenizer

T5TokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_i

In [19]:
# add sep token
tokenizer.add_special_tokens({'sep_token': '[SEP]'})

1

In [20]:
tokenizer

T5TokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<ex

In [21]:
if args.model_type == 'encoder-decoder':
    #global_attention_first_token = False  # should be True for LED
    encode_plus_kwargs = {'truncation': True, 'padding': 'longest', 'pad_to_multiple_of': 1}
    # generate_kwargs = {'max_length': args.target_seq_len, 'min_length': args.target_seq_len}
    generate_kwargs = {}

In [22]:
train_df.head()

,id,verbalization,head,tail,relation,verbalized_tail
0,0,predict [SEP] Kukavičko Lake instance of [SEP],Q6442440,Q23397,P31,lake
1,1,predict [SEP] lake inverse of instance of [SEP],Q23397,Q6442440,inverse of P31,Kukavičko Lake
2,2,"predict [SEP] Pattersonville, Ohio country [SEP]",Q7148490,Q30,P17,United States of America
3,3,predict [SEP] United States of America inverse...,Q30,Q7148490,inverse of P17,"Pattersonville, Ohio"
4,4,predict [SEP] Johnny Hughes place of birth [SEP],Q16145304,Q3752988,P19,Mountbellew


In [23]:
def collate_fn(batch):
            # print('batch', batch[0].keys(), batch[0]['input'])
            # cut too long strings because they may slow down tokenization
            inputs = [b['input'][:args.input_seq_len * 10] for b in batch] #default input_seq_len = 128
            if 'outputs' in batch[0]:
                # if we have more than 1 label per example (only in valid) take only one of them
                # to compute loss on valid
                labels = [b['outputs'][:args.target_seq_len * 10] for b in batch] #default target_seq_len = 16
            else:
                labels = [b['output'][:args.target_seq_len * 10] for b in batch]

            if args.input_prefix: #add task prefix to an input string, default = ""
                inputs = [args.input_prefix + inp for inp in inputs]


            #tokenize inputs
            features = tokenizer.batch_encode_plus(list(inputs), max_length=args.input_seq_len, return_tensors='pt',
                                                   **encode_plus_kwargs)
            #{'input_ids': [27, 8, 3, 9, 1695, 1523, 13, 8, 3, 27168, 5], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


            #tokenize labels
            with tokenizer.as_target_tokenizer():
                labels = tokenizer.batch_encode_plus(list(labels), max_length=args.target_seq_len, return_tensors='pt',
                                                     **encode_plus_kwargs).input_ids
            labels[labels == tokenizer.pad_token_id] = -100
            features['labels'] = labels

            #             features = {
            #     'input_ids': Tensor([...]),
            #     'attention_mask': Tensor([...]),
            #     'labels': Tensor([...])
            # }


            if 'outputs' in batch[0]:
                features['target_text'] = [b['outputs'] for b in batch]
            else:
                features['target_text'] = [b['output'] for b in batch]
            # if 'global_attention_mask' in features:
            #     raise RuntimeError('What global attention mask for Longformer and LongformerEncoder-Decoder should be?')
            return features

#             features = {
#     'input_ids': Tensor([...]),         # توکن‌های ورودی
#     'attention_mask': Tensor([...]),    # ماسک توجه ورودی‌ها
#     'labels': Tensor([...]),            # توکن‌های برچسب‌ها
#     'target_text': ["This is target text 1", "This is target text 2"]  # متن برچسب‌ها
# }

In [24]:
batch = [{'input': 'predict [SEP] Kukavičko Lake instance of [SEP]', 'outputs': 'lake'},{'input':'predict [SEP] lake inverse of instance of [SEP]','outputs': 'Kukavičko Lake'}]

In [ ]:
collate_fn(batch)

/home/ahmadi/sadaf/GraphNeighborLM/sadafenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': tensor([[ 9689, 32100,  3695,   157,  2960,     2,   157,    32,  2154,  3421,
            13, 32100,     1],
        [ 9689, 32100,  6957,     3, 23536,    13,  3421,    13, 32100,     1,
             0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]), 'labels': tensor([[6957,    1, -100, -100, -100, -100, -100, -100],
        [3695,  157, 2960,    2,  157,   32, 2154,    1]]), 'target_text': ['lake', 'Kukavičko Lake']}

In [26]:
logger.info(f'Preparing dataset for: {args.task_name}')

2025-01-16 15:26:35,570 - __main__ - INFO - Preparing dataset for: None


In [27]:
train_dataset = KGLMDataset(train_df, neighborhood=not args.drop_neighborhood)#it's time to use train dataset for model




In [28]:
per_worker_batch_size = args.batch_size * args.gradient_accumulation_steps
per_worker_batch_size

1

In [29]:
kwargs = {'pin_memory': True, 'num_workers': args.data_n_workers}
train_dataloader = DataLoader(
train_dataset,
batch_size=per_worker_batch_size,
shuffle=True,
collate_fn=collate_fn,
**kwargs
)

In [30]:
kwargs

{'pin_memory': True, 'num_workers': 2}

In [31]:
args.model_cls ="transformers:T5ForConditionalGeneration"

In [32]:
model_cls = get_cls_by_name(args.model_cls)  # "transformers:T5ForConditionalGeneration"
logger.info(f'Using model class: {model_cls}')
logger.info(f'Loading pretrained model: {args.from_pretrained}')
model = model_cls.from_pretrained(args.from_pretrained)

2025-01-16 15:26:40,280 - __main__ - INFO - Using model class: <class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>
2025-01-16 15:26:40,282 - __main__ - INFO - Loading pretrained model: t5-small
/home/ahmadi/sadaf/GraphNeighborLM/sadafenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
model_cls

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

In [34]:
## load cpt
if args.cpt_path: #loading best model wight from checkpoint
    model_cpt = os.path.join(args.cpt_path, "model_best.pth")
    cpt = torch.load(model_cpt, map_location='cpu')
    model.load_state_dict(cpt['model_state_dict'])


In [35]:
model.parameters()

<generator object Module.parameters at 0x7f16d0a47760>

In [36]:
lr=args.lr
lr

0.0001

In [37]:
weight_decay=args.weight_decay
weight_decay

0.0

In [38]:
logger.info('Using AdamW optimizer')
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-4 ,
    weight_decay=args.weight_decay
    )
optimizer

2025-01-16 15:26:44,418 - __main__ - INFO - Using AdamW optimizer


AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.0
)

In [39]:
def keep_for_metrics_fn(batch, output):
    # select data from batch and model output that would be used to compute metrics
    data = {}
    if 'generation_outputs' in output:
        data['labels'] = batch['target_text']  # برچسب‌های اصلی (متن هدف)
        data['generation_outputs'] = output['generation_outputs']  # متن تولیدشده توسط مدل
    return data

In [40]:
batch = collate_fn(batch)
batch

{'input_ids': tensor([[ 9689, 32100,  3695,   157,  2960,     2,   157,    32,  2154,  3421,
            13, 32100,     1],
        [ 9689, 32100,  6957,     3, 23536,    13,  3421,    13, 32100,     1,
             0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]), 'labels': tensor([[6957,    1, -100, -100, -100, -100, -100, -100],
        [3695,  157, 2960,    2,  157,   32, 2154,    1]]), 'target_text': ['lake', 'Kukavičko Lake']}

In [41]:
output = {
    "generation_outputs": ["lake","fake"]  # خروجی مدل
}

In [42]:
keep_for_metrics_fn(batch, output)

{'labels': ['lake', 'Kukavičko Lake'], 'generation_outputs': ['lake', 'fake']}

In [43]:
data ={'labels': ['lake', 'Kukavičko Lake'], 'generation_outputs': ['lake', 'fake']}

In [174]:
def metrics_fn(data):
    # compute metrics based on stored labels, predictions, ...
    metrics = {}
    y, p = None, None

    if args.model_type == 'encoder-decoder' and 'generation_outputs' in data:
        # replace -100 with pad token in labels
        y = data['labels']
        # print('!', data['generation_outputs'].shape)
        p = tokenizer.batch_decode(data['generation_outputs'], skip_special_tokens=True)
        if args.show_valid_examples > 0:
        # if args.show_valid_examples > 0:
            for i in range(min(args.show_valid_examples, len(y))):
                logger.info(f'y: {y[i]}')
                logger.info(f'p: {p[i]}')
                logger.info(f'p ids: {data["generation_outputs"][i]}')
                logger.info('-' * 50)


    if y is not None and p is not None:
        metrics['exact_match'] = accuracy_score(y, p) * 100

    return metrics

In [175]:
metrics_fn(data)

TypeError: argument 'ids': Can't extract `str` to `Vec`

In [176]:

valid_dataset = KGLMDataset(valid_df, neighborhood=not args.drop_neighborhood)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=per_worker_batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    **kwargs
)

In [177]:
# داده‌های نمونه
data = {'labels': ['lake', 'Kukavičko Lake'], 'generation_outputs': ['lake', 'fake']}

# تبدیل generation_outputs به توکن‌ها
data['generation_outputs'] = [
    tokenizer.encode(output, add_special_tokens=True) for output in data['generation_outputs']
]

# تعریف آرگومان‌ها
class Args:
    model_type = 'encoder-decoder'
    show_valid_examples = 2


# تعریف تابع metrics_fn
def metrics_fn(data):
    metrics = {}
    y, p = None, None

    if args.model_type == 'encoder-decoder' and 'generation_outputs' in data:
        y = data['labels']
        p = tokenizer.batch_decode(data['generation_outputs'], skip_special_tokens=True)
        if args.show_valid_examples > 0:
            for i in range(min(args.show_valid_examples, len(y))):
                logger.info(f'y: {y[i]}')
                logger.info(f'p: {p[i]}')
                logger.info(f'p ids: {data["generation_outputs"][i]}')
                logger.info('-' * 50)

    if y is not None and p is not None:
        metrics['exact_match'] = accuracy_score(y, p) * 100

    return metrics

# تست تابع
metrics = metrics_fn(data)
print("Metrics:", metrics)

2025-01-16 10:06:10,739 - __main__ - INFO - y: lake
2025-01-16 10:06:10,741 - __main__ - INFO - p: lake
2025-01-16 10:06:10,742 - __main__ - INFO - p ids: [6957, 1]
2025-01-16 10:06:10,743 - __main__ - INFO - --------------------------------------------------
2025-01-16 10:06:10,745 - __main__ - INFO - y: Kukavičko Lake
2025-01-16 10:06:10,746 - __main__ - INFO - p: fake
2025-01-16 10:06:10,747 - __main__ - INFO - p ids: [9901, 1]
2025-01-16 10:06:10,748 - __main__ - INFO - --------------------------------------------------


Metrics: {'exact_match': 50.0}


In [178]:
metrics

{'exact_match': 50.0}

In [166]:
args.lr_scheduler = None

In [17]:
if __name__ == '__main__':
    import sys
    sys.argv = ['']  # حذف آرگومان‌های اضافی Jupyter
    args = parser.parse_args()
    logger.info('Running in environment.')
    logger.info('Using a single GPU setup.')
    logger.info(f'FP16 training is set to: {args.fp16}') #default = false

    if args.model_path is None: # path where to save model, default: None
      logger.warning('model_path is not set: config, logs and checkpoints will not be saved.')

    # create model path and save configuration
    if args.model_path is not None:
        model_path = Path(args.model_path)

        if not model_path.exists():
            Path(model_path).mkdir(parents=True)
        args_dict = collect_run_configuration(args)
        # todo: if model path exists and there is config file, write new config file aside
        json.dump(args_dict, open(model_path/'config.json', 'w'), indent=4)

 #output is like below:
#         {
#     'batch_size': 32,
#     'learning_rate': 0.001,
#     'model': 'T5',
#     'ENV': {
#         'CUDA_VISIBLE_DEVICES': '0,1'
#     },
#     'MACHINE': 'colab-instance',
#     'COMMIT': '3fa4b7c2c8f67a9d8e45be68dca1e24ff8b524d1'
# }


    if args.tokenizer:
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer) #sepcialized tokenizer
    else:
        tokenizer = AutoTokenizer.from_pretrained(args.from_pretrained) #general tokenizer-->t5-small


    # add sep token
    tokenizer.add_special_tokens({'sep_token': '[SEP]'})

    if args.model_type == 'encoder-decoder':
        #global_attention_first_token = False  # should be True for LED
        encode_plus_kwargs = {'truncation': True, 'padding': 'longest', 'pad_to_multiple_of': 1}
        # generate_kwargs = {'max_length': args.target_seq_len, 'min_length': args.target_seq_len}
        generate_kwargs = {}


        def collate_fn(batch):
            # print('batch', batch[0].keys(), batch[0]['input'])
            # cut too long strings because they may slow down tokenization
            inputs = [b['input'][:args.input_seq_len * 10] for b in batch] #default input_seq_len = 128
            if 'outputs' in batch[0]:
                # if we have more than 1 label per example (only in valid) take only one of them
                # to compute loss on valid
                labels = [b['outputs'][:args.target_seq_len * 10] for b in batch] #default target_seq_len = 16
            else:
                labels = [b['output'][:args.target_seq_len * 10] for b in batch]

            if args.input_prefix: #add task prefix to an input string, default = ""
                inputs = [args.input_prefix + inp for inp in inputs]


            #tokenize inputs
            features = tokenizer.batch_encode_plus(list(inputs), max_length=args.input_seq_len, return_tensors='pt',
                                                   **encode_plus_kwargs)
            #{'input_ids': [27, 8, 3, 9, 1695, 1523, 13, 8, 3, 27168, 5], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


            #tokenize labels
            with tokenizer.as_target_tokenizer():
                labels = tokenizer.batch_encode_plus(list(labels), max_length=args.target_seq_len, return_tensors='pt',
                                                     **encode_plus_kwargs).input_ids
            labels[labels == tokenizer.pad_token_id] = -100
            features['labels'] = labels

            #             features = {
            #     'input_ids': Tensor([...]),
            #     'attention_mask': Tensor([...]),
            #     'labels': Tensor([...])
            # }


            if 'outputs' in batch[0]:
                features['target_text'] = [b['outputs'] for b in batch]
            else:
                features['target_text'] = [b['output'] for b in batch]
            # if 'global_attention_mask' in features:
            #     raise RuntimeError('What global attention mask for Longformer and LongformerEncoder-Decoder should be?')
            return features

#             features = {
#     'input_ids': Tensor([...]),         # توکن‌های ورودی
#     'attention_mask': Tensor([...]),    # ماسک توجه ورودی‌ها
#     'labels': Tensor([...]),            # توکن‌های برچسب‌ها
#     'target_text': ["This is target text 1", "This is target text 2"]  # متن برچسب‌ها
# }

    logger.info(f'Preparing dataset for: {args.task_name}')

    train_dataset = KGLMDataset(train_df, neighborhood=not args.drop_neighborhood)#it's time to use train dataset for model

    per_worker_batch_size = args.batch_size * args.gradient_accumulation_steps
    kwargs = {'pin_memory': True, 'num_workers': args.data_n_workers}
    train_dataloader = DataLoader(
    train_dataset,
    batch_size=per_worker_batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    **kwargs
)

    logger.info(f'Preparing validation data for: {args.task_name}')
    valid_dataset = KGLMDataset(valid_df, neighborhood=not args.drop_neighborhood)
    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=per_worker_batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        **kwargs
    )

    #log on validation data every N batches (default: None)
    if args.valid_interval is None:

      args.valid_interval = args.log_interval  #log to report loss, metrics on training data every N batches (default: None)



    model_cls = get_cls_by_name(args.model_cls)  # "transformers:T5ForConditionalGeneration"
    logger.info(f'Using model class: {model_cls}')
    logger.info(f'Loading pretrained model: {args.from_pretrained}')
    model = model_cls.from_pretrained(args.from_pretrained)

    ## load cpt
    if args.cpt_path: #loading best model wight from checkpoint
        model_cpt = os.path.join(args.cpt_path, "model_best.pth")
        cpt = torch.load(model_cpt, map_location='cpu')
        model.load_state_dict(cpt['model_state_dict'])



    logger.info('Using AdamW optimizer')
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=1e-4 ,
        weight_decay=args.weight_decay
        )


    def keep_for_metrics_fn(batch, output):
      # select data from batch and model output that would be used to compute metrics
      data = {}
      if 'generation_outputs' in output:
          data['labels'] = batch['target_text']  # برچسب‌های اصلی (متن هدف)
          data['generation_outputs'] = output['generation_outputs']  # متن تولیدشده توسط مدل
      return data


    def metrics_fn(data):
      # compute metrics based on stored labels, predictions, ...
      metrics = {}
      y, p = None, None

      if args.model_type == 'encoder-decoder' and 'generation_outputs' in data:
          # replace -100 with pad token in labels
          y = data['labels']
          # print('!', data['generation_outputs'].shape)
          p = tokenizer.batch_decode(data['generation_outputs'], skip_special_tokens=True)
          if args.show_valid_examples > 0:
          # if args.show_valid_examples > 0:
              for i in range(min(args.show_valid_examples, len(y))):
                  logger.info(f'y: {y[i]}')
                  logger.info(f'p: {p[i]}')
                  logger.info(f'p ids: {data["generation_outputs"][i]}')
                  logger.info('-' * 50)


      if y is not None and p is not None:
          metrics['exact_match'] = accuracy_score(y, p) * 100

      return metrics

    trainer = Trainer(args, model, optimizer, train_dataloader, valid_dataloader, 
                      keep_for_metrics_fn=keep_for_metrics_fn, metrics_fn=metrics_fn,
                      generate_kwargs=generate_kwargs if args.use_generate_on_valid else {})

    if not args.validate_only:
        # train loop
        trainer.train()
        # make sure all workers are done
  
        # run validation after training
        if args.save_best:
            best_model_path = str(Path(args.model_path) / 'model_best.pth')

            logger.info(f'Loading best saved model from {best_model_path}')
            trainer.load(best_model_path)
        if valid_dataloader is not None:

            logger.info('Runnning validation on valid data:')
            trainer.validate(valid_dataloader, write_tb=False)
    else:
        # run validation, do not write to tensorboard

        logger.info('Running validation on train set:')
        trainer.validate(train_dataloader, split='train', write_tb=False)
        if valid_dataloader is not None:

            logger.info('Running validation on valid data:')
            trainer.validate(valid_dataloader, write_tb=False)


2025-01-16 16:20:59,936 - __main__ - INFO - Running in environment.
2025-01-16 16:20:59,938 - __main__ - INFO - Using a single GPU setup.
2025-01-16 16:20:59,939 - __main__ - INFO - FP16 training is set to: False


/home/ahmadi/sadaf/GraphNeighborLM/sadafenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2025-01-16 16:21:00,384 - __main__ - INFO - Preparing dataset for: None
2025-01-16 16:21:00,385 - __main__ - INFO - Preparing validation data for: None
2025-01-16 16:21:00,386 - __main__ - INFO - Using model class: <class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>
2025-01-16 16:21:00,387 - __main__ - INFO - Loading pretrained model: t5-small
2025-01-16 16:21:01,919 - __main__ - INFO - Using AdamW optimizer

/home/ahmadi/sadaf/GraphNeighborLM/sadafenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using th

In [151]:
args.valid_interval